<a href="https://colab.research.google.com/github/Suwandi-amin-sangaji/modelVGG16_yuyu/blob/main/YuyuVgg16baru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, Lambda, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import cv2

os.listdir("/content/drive/MyDrive/dataset_obat_yuyu")

print("Setup Complete!")

In [ ]:
IMG_SIZE = 64

category = ['Ambroxol_Tablet', 'Diapet_Kapsul', 'Flamar_50_Tablet','Folavit_Tablet', 'Laxing_Kapsul', 'Meftormin_Tablet','Omeprazole_Kapsul','Ranitidine_Tablet','Sangobion_Kapsul','Simcobal_Kapsul']
def get_train_data(direct):
    data = []
    for labels in category:
        path = os.path.join(direct, labels)
        class_num = category.index(labels)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                data.append([new_array, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [ ]:
new_data = get_train_data("/content/drive/MyDrive/dataset_obat_yuyu")

In [ ]:
X = []
y = []
for feature, label in new_data:
    X.append(feature)
    y.append(label)
    
from keras.utils import np_utils
y = np_utils.to_categorical(y, num_classes = 10)

X = np.array(X)
y = np.array(y)

X = X.reshape(-1, 64, 64, 3)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rescale = 1/255,
                                     zoom_range = 0.3,
                                     horizontal_flip = True,
                                     rotation_range = 30)

test_generator = ImageDataGenerator(rescale = 1/255)

train_generator = train_generator.flow(np.array(X_train),
                                       y_train,
                                       batch_size = 64,
                                       shuffle = False)

test_generator = test_generator.flow(np.array(X_test),
                                     y_test,
                                     batch_size = 64,
                                     shuffle = False)

print("Train data shape: ", X_train.shape)
print("Train labels shape: ", y_train.shape)
print("Test data shape: ", X_test.shape)
print("Test labels shape: ", y_test.shape)

In [ ]:
vg16 = VGG16(input_shape=[IMG_SIZE, IMG_SIZE] + [3], weights="imagenet", include_top=False)

for layer in vg16.layers:
    layer.trainable = False

x = Flatten()(vg16.output)
prediction = Dense(len(category), activation="softmax")(x)
model = Model(inputs=vg16.input, outputs=prediction)


model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

In [ ]:
!pip install visualkeras

In [ ]:
import visualkeras
visualkeras.layered_view(model)

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='modelvgg16yuyu.png',show_shapes=True)

In [ ]:
history = model.fit_generator(train_generator,
                                   steps_per_epoch = len(X_train)/64,
                                   epochs = 50,
                                   shuffle = False,
                                   validation_data=test_generator, validation_steps=len(test_generator))

In [ ]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))

plt.plot(epochs, accuracy, "b", label="trainning accuracy")
plt.plot(epochs, val_accuracy, "r", label="validation accuracy")
plt.legend()
plt.show()

plt.plot(epochs, loss, "b", label="trainning loss")
plt.plot(epochs, val_loss, "r", label="validation loss")
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.preprocessing import image
output_class = ['Ambroxol_Tablet', 'Diapet_Kapsul', 'Flamar_50_Tablet','Folavit_Tablet', 'Laxing_Kapsul', 'Meftormin_Tablet','Omeprazole_Kapsul','Ranitidine_Tablet','Sangobion_Kapsul','Simcobal_Kapsul']
def obat(new_image):
  test_image = image.load_img(new_image, target_size = (64,64))
  plt.axis("off")
  plt.imshow(test_image)
  plt.show()
 
  test_image = image.img_to_array(test_image) / 255
  test_image = np.expand_dims(test_image, axis=0)

  predicted_array = model.predict(test_image)
  predicted_value = output_class[np.argmax(predicted_array)]
  predicted_accuracy = round(np.max(predicted_array) * 100, 2)

  print("Obat is ", predicted_value, " with ", predicted_accuracy, " % accuracy")

In [ ]:
obat('/content/drive/MyDrive/dataset_obat_yuyu/Ambroxol_Tablet/IMG_5408.jpg')

In [ ]:
import tensorflow as tf
from keras.models import load_model
model.save('//content/drive/MyDrive/dataset_obat_yuyu/yuyumodel.h5')

In [ ]:
y_pred = model.predict(test_generator)
y_pred

In [ ]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

In [ ]:
# from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=category))

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Generate the classification report
report = classification_report(np.argmax(y_test, axis=1), y_pred, target_names=category, output_dict=True)

# Print the report with a nice formatting
print("{:<20} {:<10} {:<10} {:<10} {:<10}".format("", "precision", "recall", "f1-score", "support"))
for key in report.keys():
    if key in category:
        values = report[key]
        print("{:<20} {:<10.2f} {:<10.2f} {:<10.2f} {:<10}".format(key, values['precision'], values['recall'], values['f1-score'], values['support']))
print("\nAccuracy: {:.2f}".format(accuracy_score(np.argmax(y_test, axis=1), y_pred)))


import seaborn as sns
import matplotlib.pyplot as plt

# Generate the confusion matrix
cm = confusion_matrix(np.argmax(y_test, axis=1), y_pred)

# Create a heatmap for the confusion matrix
fig, ax = plt.subplots(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', cbar=False, ax=ax, xticklabels=category, yticklabels=category)
ax.set_xlabel('Predicted Labels')
ax.set_ylabel('True Labels')
plt.show()

